# Preprocessing
Here we merge the 5-minute returns data and the open and close price data.

In [1]:
import numpy as np
import pandas as pd

/Users/luuk/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
dataHF = pd.read_csv("/Users/luuk/Documenten/School/Master/Research Assistantship/Data/HFData5DJIA3Y.csv")       #5-minute returns
dataOC = pd.read_csv("/Users/luuk/Documenten/School/Master/Research Assistantship/Data/OpenCloseData3YDJIA.csv")     #Open and close price data

data = dataHF[["SYMBOL", "PRICE", "DATE", "DT"]]
data["TIME"] = data["DT"].apply(lambda str: str.split()[-1])
data = data.drop(columns=["DT"])

/var/folders/js/sz7lv_rx4tz_t0fgb1g89k440000gn/T/ipykernel_18924/3895306201.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dataHF = pd.read_csv("/Users/luuk/Documenten/School/Master/Research Assistantship/Data/HFData5DJIA3Y.csv")       #5-minute returns


,SYMBOL,PRICE,DATE,DT
0,MMM,177.54,2020-01-02,2020-01-02 09:30:00
1,MMM,177.54,2020-01-02,2020-01-02 09:35:00
2,MMM,178.43,2020-01-02,2020-01-02 09:40:00
3,MMM,179.19,2020-01-02,2020-01-02 09:45:00
4,MMM,178.85,2020-01-02,2020-01-02 09:50:00
...,...,...,...,...
2977142,DIS,85.96,2022-12-30,2022-12-30 15:40:00
2977143,DIS,86.16,2022-12-30,2022-12-30 15:45:00
2977144,DIS,86.41,2022-12-30,2022-12-30 15:50:00
2977145,DIS,86.75,2022-12-30,2022-12-30 15:55:00


In [6]:
data

,SYMBOL,PRICE,DATE,DT,TIME
0,MMM,177.54,2020-01-02,2020-01-02 09:30:00,09:30:00
1,MMM,177.54,2020-01-02,2020-01-02 09:35:00,09:35:00
2,MMM,178.43,2020-01-02,2020-01-02 09:40:00,09:40:00
3,MMM,179.19,2020-01-02,2020-01-02 09:45:00,09:45:00
4,MMM,178.85,2020-01-02,2020-01-02 09:50:00,09:50:00
...,...,...,...,...,...
2977142,DIS,85.96,2022-12-30,2022-12-30 15:40:00,15:40:00
2977143,DIS,86.16,2022-12-30,2022-12-30 15:45:00,15:45:00
2977144,DIS,86.41,2022-12-30,2022-12-30 15:50:00,15:50:00
2977145,DIS,86.75,2022-12-30,2022-12-30 15:55:00,15:55:00


In [7]:
dates = np.unique(data["DATE"])
stocks = np.unique(data["SYMBOL"])

nMissing = 0        #Initialize the number of missing observations to 0
for date in dates:
    nobsDate = len(data[(data["SYMBOL"] == "AAPL") & (data["DATE"] == date)])        #Extract how many observations we have for each trading date (some trading days end at 13:00)
    timesForDate = np.unique(data[(data["SYMBOL"] == "AAPL") & (data["DATE"] == date)]["TIME"])
    
    for stock in stocks:
        filteredData = data[(data["SYMBOL"] == stock) & (data["DATE"] == date)]     #Consider only the observations for the particular stock on the particular date
        indeces = filteredData.index
        
        data.loc[indeces[0], "PRICE"] = dataOC.loc[(dataOC["Ticker"] == stock) & (dataOC["DlyCalDt"] == date), "DlyOpen"].values[0]     #Replace the opening price
        data.loc[indeces[-1], "PRICE"] = dataOC.loc[(dataOC["Ticker"] == stock) & (dataOC["DlyCalDt"] == date), "DlyClose"].values[0]     #Replace the closing price
        
        if (len(filteredData) < nobsDate):      #Check for missing observations
            t = 0
            
            for time in timesForDate:
                if not (time in filteredData["TIME"].values):
                    nMissing += 1
                    
                    idxPrev = data[(data["SYMBOL"] == stock) & (data["DATE"] == date) & (data["TIME"] == timesForDate[t-1])].index[0]       #Extract the index of the observation from 5 minutes before
                    previousPrice = data.loc[idxPrev, "PRICE"]
                    newRow = pd.DataFrame({"SYMBOL": stock, "PRICE": previousPrice, "DATE": date, "TIME": timesForDate[t]}, index=[idxPrev+1])      #Instead of a missing observation we use the price from 5 minutes before
                    
                    data = pd.concat([data[:idxPrev], newRow, data[idxPrev:]]).reset_index(drop=True)
                
                t += 1
    print(date)         #Print the date to indicate how fast the process is going

2020-01-02
2020-01-03
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-21
2020-01-22
2020-01-23
2020-01-24
2020-01-27
2020-01-28
2020-01-29
2020-01-30
2020-01-31
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-30
2020-03-31
2020-04-01
2020-04-02
2020-04-03
2020-04-06
2020-04-07
2020-04-08
2020-04-09
2020-04-13
2020-04-14
2020-04-15
2020-04-16
2020-04-17
2020-04-20
2020-04-21
2020-04-22
2020-04-23
2020-04-24
2020-04-27
2020-04-28
2020-04-29
2020-04-30
2020-05-01
2020-05-04
2020-05-05
2020-05-06
2020-05-07
2020-05-08
2020-05-11
2020-05-12

In [8]:
data

,SYMBOL,PRICE,DATE,DT,TIME
0,MMM,177.68,2020-01-02,2020-01-02 09:30:00,09:30:00
1,MMM,177.54,2020-01-02,2020-01-02 09:35:00,09:35:00
2,MMM,178.43,2020-01-02,2020-01-02 09:40:00,09:40:00
3,MMM,179.19,2020-01-02,2020-01-02 09:45:00,09:45:00
4,MMM,178.85,2020-01-02,2020-01-02 09:50:00,09:50:00
...,...,...,...,...,...
2977227,DIS,85.96,2022-12-30,2022-12-30 15:40:00,15:40:00
2977228,DIS,86.16,2022-12-30,2022-12-30 15:45:00,15:45:00
2977229,DIS,86.41,2022-12-30,2022-12-30 15:50:00,15:50:00
2977230,DIS,86.75,2022-12-30,2022-12-30 15:55:00,15:55:00


In [9]:
data = data.drop_duplicates(subset=["SYMBOL", "DATE", "TIME"])

In [10]:
data

,SYMBOL,PRICE,DATE,DT,TIME
0,MMM,177.68,2020-01-02,2020-01-02 09:30:00,09:30:00
1,MMM,177.54,2020-01-02,2020-01-02 09:35:00,09:35:00
2,MMM,178.43,2020-01-02,2020-01-02 09:40:00,09:40:00
3,MMM,179.19,2020-01-02,2020-01-02 09:45:00,09:45:00
4,MMM,178.85,2020-01-02,2020-01-02 09:50:00,09:50:00
...,...,...,...,...,...
2957434,DIS,85.96,2022-12-30,2022-12-30 15:40:00,15:40:00
2957435,DIS,86.16,2022-12-30,2022-12-30 15:45:00,15:45:00
2957436,DIS,86.41,2022-12-30,2022-12-30 15:50:00,15:50:00
2957437,DIS,86.75,2022-12-30,2022-12-30 15:55:00,15:55:00


In [11]:
nMissing

85

In [12]:
data.to_csv("FinalData5.csv")